In [2]:
import duckdb
import boto3
import os
# Configure S3 connection
MINIO_ENDPOINT = os.environ.get("S3_ENDPOINT", "http://minio:9000")
MINIO_ACCESS_KEY = os.environ.get("S3_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("S3_SECRET_KEY", "minioadmin")
s3_region = os.environ.get("S3_REGION", "us-east-1")
BUCKET_NAME = "bronze"
PARQUET_FOLDER = "breweries"  # Folder inside the bucket

# Initialize MinIO S3 client
s3_client = boto3.client(
    "s3",
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
)

# List all Parquet files in MinIO bucket
def list_parquet_files(bucket, folder):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder)
    if "Contents" in response:
        return [obj["Key"] for obj in response["Contents"] if obj["Key"].endswith(".parquet")]
    return []

# Generate proper S3 URLs for DuckDB
def generate_s3_urls(bucket, parquet_files):
    return [f"s3://{bucket}/{file}" for file in parquet_files]

# Get list of parquet files
parquet_files = list_parquet_files(BUCKET_NAME, PARQUET_FOLDER)
if not parquet_files:
    print("No Parquet files found.")
    exit()

# Convert to S3 URLs
s3_urls = generate_s3_urls(BUCKET_NAME, parquet_files)
print(s3_urls)
# Connect DuckDB and enable S3 support
con = duckdb.connect(database=":memory:")
con.execute(f"""
    INSTALL httpfs;
    LOAD httpfs;
SET s3_region='us-east-1';
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
SET s3_endpoint='minio:9000';
SET s3_url_style='path';
""")

# Query the Parquet files in MinIO
query = f"""
    SELECT * FROM parquet_scan({s3_urls})
"""
df = con.execute(query).fetchdf()

['s3://bronze/breweries/part-00000-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet', 's3://bronze/breweries/part-00000-7c2161ae-085c-4df9-83d6-464ff688c0e1-c000.snappy.parquet', 's3://bronze/breweries/part-00000-bd4bf2af-f6c8-4564-a7b0-9cc26fda9225-c000.snappy.parquet', 's3://bronze/breweries/part-00001-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet', 's3://bronze/breweries/part-00001-7c2161ae-085c-4df9-83d6-464ff688c0e1-c000.snappy.parquet', 's3://bronze/breweries/part-00001-bd4bf2af-f6c8-4564-a7b0-9cc26fda9225-c000.snappy.parquet', 's3://bronze/breweries/part-00002-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet', 's3://bronze/breweries/part-00002-7c2161ae-085c-4df9-83d6-464ff688c0e1-c000.snappy.parquet', 's3://bronze/breweries/part-00002-bd4bf2af-f6c8-4564-a7b0-9cc26fda9225-c000.snappy.parquet', 's3://bronze/breweries/part-00003-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet', 's3://bronze/breweries/part-00003-7c2161ae-085c-4df9-83d6-464ff688c0e

IOException: IO Error: SSLConnection error for HTTP HEAD to 'https://minio:9000/bronze/breweries/part-00000-17daba81-b7da-44ad-bcb0-352a30d48a3f-c000.snappy.parquet'

In [7]:
# List all Parquet files in MinIO bucket
def list_parquet_files(bucket, folder):
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder)
    if "Contents" in response:
        return [obj["Key"] for obj in response["Contents"] if obj["Key"].endswith(".parquet")]
    return []

# Generate MinIO S3 URLs for DuckDB
def generate_s3_urls(bucket, parquet_files):
    return [f"s3://{bucket}/{file}" for file in parquet_files]
BUCKET_NAME = 'bronze'
PARQUET_FOLDER = 'breweries'
# Get list of parquet files
parquet_files = list_parquet_files(BUCKET_NAME, PARQUET_FOLDER)


NameError: name 's3_client' is not defined